In [ ]:
!pip -q install "ibis-framework[duckdb,examples]" duckdb pyarrow pandas

import ibis
from ibis import _

print("Ibis version:", ibis.__version__)

con = ibis.duckdb.connect()
ibis.options.interactive = True

In [ ]:
try:
    base_expr = ibis.examples.penguins.fetch(backend=con)
except TypeError:
    base_expr = ibis.examples.penguins.fetch()

if "penguins" not in con.list_tables():
    try:
        con.create_table("penguins", base_expr, overwrite=True)
    except Exception:
        con.create_table("penguins", base_expr.execute(), overwrite=True)

t = con.table("penguins")
print(t.schema())

In [ ]:
def penguin_feature_pipeline(penguins):
    base = penguins.mutate(
        bill_ratio=_.bill_length_mm / _.bill_depth_mm,
        is_male=(_.sex == "male").ifelse(1, 0),
    )

    cleaned = base.filter(
        _.bill_length_mm.notnull()
        & _.bill_depth_mm.notnull()
        & _.body_mass_g.notnull()
        & _.flipper_length_mm.notnull()
        & _.species.notnull()
        & _.island.notnull()
        & _.year.notnull()
    )

    w_species = ibis.window(group_by=[cleaned.species])
    w_island_year = ibis.window(
        group_by=[cleaned.island],
        order_by=[cleaned.year],
        preceding=2,
        following=0,
    )

    feat = cleaned.mutate(
        species_avg_mass=cleaned.body_mass_g.mean().over(w_species),
        species_std_mass=cleaned.body_mass_g.std().over(w_species),
        mass_z=(
            cleaned.body_mass_g
            - cleaned.body_mass_g.mean().over(w_species)
        ) / cleaned.body_mass_g.std().over(w_species),
        island_mass_rank=cleaned.body_mass_g.rank().over(
            ibis.window(group_by=[cleaned.island])
        ),
        rolling_3yr_island_avg_mass=cleaned.body_mass_g.mean().over(
            w_island_year
        ),
    )

    return feat.group_by(["species", "island", "year"]).agg(
        n=feat.count(),
        avg_mass=feat.body_mass_g.mean(),
        avg_flipper=feat.flipper_length_mm.mean(),
        avg_bill_ratio=feat.bill_ratio.mean(),
        avg_mass_z=feat.mass_z.mean(),
        avg_rolling_3yr_mass=feat.rolling_3yr_island_avg_mass.mean(),
        pct_male=feat.is_male.mean(),
    ).order_by(["species", "island", "year"])

In [ ]:
features = penguin_feature_pipeline(t)
print(con.compile(features))

try:
    df = features.to_pandas()
except Exception:
    df = features.execute()

display(df.head())

In [5]:
con.create_table("penguin_features", features, overwrite=True)

feat_tbl = con.table("penguin_features")

try:
    preview = feat_tbl.limit(10).to_pandas()
except Exception:
    preview = feat_tbl.limit(10).execute()

display(preview)

out_path = "/content/penguin_features.parquet"
con.raw_sql(f"COPY penguin_features TO '{out_path}' (FORMAT PARQUET);")
print(out_path)

Ibis version: 9.5.0
Base table schema (in DuckDB):
ibis.Schema {
  species            string
  island             string
  bill_length_mm     float64
  bill_depth_mm      float64
  flipper_length_mm  int64
  body_mass_g        int64
  sex                string
  year               int64
}
DuckDB tables: ['penguins']

--- Compiled DuckDB SQL ---

SELECT * FROM (SELECT "t2"."species", "t2"."island", "t2"."year", COUNT(*) AS "n", AVG("t2"."body_mass_g") AS "avg_mass", AVG("t2"."flipper_length_mm") AS "avg_flipper", AVG("t2"."bill_ratio") AS "avg_bill_ratio", AVG("t2"."mass_z") AS "avg_mass_z", AVG("t2"."rolling_3yr_island_avg_mass") AS "avg_rolling_3yr_mass", AVG("t2"."is_male") AS "pct_male" FROM (SELECT "t1"."species", "t1"."island", "t1"."bill_length_mm", "t1"."bill_depth_mm", "t1"."flipper_length_mm", "t1"."body_mass_g", "t1"."sex", "t1"."year", "t1"."bill_ratio", "t1"."is_male", AVG("t1"."body_mass_g") OVER (PARTITION BY "t1"."species" ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED F

,species,island,year,n,avg_mass,avg_flipper,avg_bill_ratio,avg_mass_z,avg_rolling_3yr_mass,pct_male
0,Adelie,Biscoe,2007,10,3620.000000,181.700000,2.081130,-0.175901,3723.333333,0.500000
1,Adelie,Biscoe,2008,18,3627.777778,189.555556,2.137682,-0.158940,3835.185185,0.500000
2,Adelie,Biscoe,2009,16,3857.812500,192.375000,2.141724,0.342699,4024.479167,0.500000
3,Adelie,Dream,2007,20,3671.250000,186.500000,2.096742,-0.064140,3631.875000,0.500000
4,Adelie,Dream,2008,16,3756.250000,192.000000,2.083660,0.121221,3761.458333,0.500000
5,Adelie,Dream,2009,20,3651.250000,191.150000,2.153120,-0.107754,3715.000000,0.500000
6,Adelie,Torgersen,2007,19,3763.157895,189.263158,2.047598,0.136285,3689.035088,0.368421
7,Adelie,Torgersen,2008,16,3856.250000,191.750000,2.145060,0.339292,3870.833333,0.500000
8,Adelie,Torgersen,2009,16,3489.062500,192.937500,2.185021,-0.461438,3533.854167,0.500000
9,Chinstrap,Dream,2007,26,3694.230769,192.423077,2.638122,-0.101103,3660.897436,0.500000



Result shape: (15, 10)

DuckDB tables after materialization: ['penguin_features', 'penguins']

--- Materialized table preview ---



,species,island,year,n,avg_mass,avg_flipper,avg_bill_ratio,avg_mass_z,avg_rolling_3yr_mass,pct_male
0,Adelie,Biscoe,2007,10,3620.000000,181.700000,2.081130,-0.175901,3723.333333,0.500000
1,Adelie,Biscoe,2008,18,3627.777778,189.555556,2.137682,-0.158940,3835.185185,0.500000
2,Adelie,Biscoe,2009,16,3857.812500,192.375000,2.141724,0.342699,4024.479167,0.500000
3,Adelie,Dream,2007,20,3671.250000,186.500000,2.096742,-0.064140,3631.875000,0.500000
4,Adelie,Dream,2008,16,3756.250000,192.000000,2.083660,0.121221,3761.458333,0.500000
5,Adelie,Dream,2009,20,3651.250000,191.150000,2.153120,-0.107754,3715.000000,0.500000
6,Adelie,Torgersen,2007,19,3763.157895,189.263158,2.047598,0.136285,3689.035088,0.368421
7,Adelie,Torgersen,2008,16,3856.250000,191.750000,2.145060,0.339292,3870.833333,0.500000
8,Adelie,Torgersen,2009,16,3489.062500,192.937500,2.185021,-0.461438,3533.854167,0.500000
9,Chinstrap,Dream,2007,26,3694.230769,192.423077,2.638122,-0.101103,3660.897436,0.500000



Saved Parquet to: /content/penguin_features.parquet

Portability tip:
- Keep `penguin_feature_pipeline()` unchanged.
- Swap the backend connection (DuckDB -> BigQuery/Snowflake/etc.).
- Compile to backend SQL and execute where the data lives.

